In [1]:
# import some common libraries
import os, warnings
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

# import tf
# import tensorflow as tf
# from tensorflow.keras import backend as K

# import image utilities
import urllib.request
from PIL import Image

import uuid

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
warnings.filterwarnings("ignore") # to clean up output cells

In [2]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("deepfashion_train",)

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65 # set threshold for this model

cfg.MODEL.WEIGHTS = "/data/deepfashion2/models/second_model/model_final.pth"
cfg.MODEL.DEVICE='cpu'
predictor = DefaultPredictor(cfg)

In [3]:
def resize_image(image):
    (h, w) = image.shape[:2]

    if w > h:
        ratio = 299.0 / w
        x = 299
        y = int(h * ratio)
        dim = (x, y)
    else:
        ratio = 299.0 / h
        y = 299
        x = int(w * ratio)
        dim = (x, y)

    # perform the actual resizing of the image
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_LINEAR)

    if resized.shape[0] > resized.shape[1]:                            
    # numpy로 검은색 배경을 생성                                            
        background = np.zeros((resized.shape[0],resized.shape[0], 3),dtype=np.uint8) 
     # 이미지를 가운데로 겹치도록 설정
        x_offset = y_offset = resized.shape[0] // 2 - (resized.shape[1] // 2)   
    # 해당 위치에 이미지를 겹침                    
        background[:, x_offset:x_offset + resized.shape[1]] = resized                              
    else:
    # 가로가 더 길 경우
        background = np.zeros((resized.shape[1], resized.shape[1], 3),dtype=np.uint8) 
        x_offset = y_offset = resized.shape[1] // 2 - (resized.shape[0] // 2)
        background[y_offset:y_offset + resized.shape[0],:] = resized
    return background

In [12]:
path = '/home/azure/ai/data/clasification/train_images'
dirs = os.listdir(path)
output_path = '/home/azure/ai/data/train_crop_images'
os.chdir(output_path)
for dir in dirs:
    if not os.path.exists(dir):
            os.makedirs(dir)

In [23]:
path = '/home/azure/ai/data/clasification/train_images'
output_path = '/home/azure/ai/data/train_crop_images'
dirs = os.listdir(path)
for dir in dirs:
    img_names = os.listdir(path + '/' + dir)
    for name in img_names:
        im = cv2.imread(path + '/' + dir + '/' + name)
        outputs = predictor(im)
    
        boxes = {}
        for coordinates in outputs["instances"].to("cpu").pred_boxes:
            coordinates_array = []
            for k in coordinates:
                coordinates_array.append(int(k))
            boxes[uuid.uuid4().hex[:].upper()] = coordinates_array
            
        name = name.replace('.jpg', '')
        j = 1    
        for k,v in boxes.items():
            crop_img = im[v[1]:v[3], v[0]:v[2], :]
            crop_img = resize_image(crop_img)
            cv2.imwrite(os.path.join(output_path + '/' + dir, name + '_' + str(j) + '.jpg'), crop_img)
            j += 1

AttributeError: 'NoneType' object has no attribute 'shape'